In [65]:
import os
import numpy as np
import pandas as pd
import statsmodels.api as sm
from typing import Any, Tuple

In [66]:
# ---------- CONFIG ----------
CSV_PATH   = "/Users/tanishq/PycharmProjects/NIScPR/FinTech/DerwentData_TS/dfa/MainData.csv"
ID_COL     = "Publication Number"
DATE_COL   = "Application Date"

INDICATORS = [
    "DWPI Count of Family Members",
    "DWPI Count of Family Countries/Regions",
    "Assignee Count",
    "Inventor Count",
    "Claims Count",
    "Legal Years Remaining",
    "IPC Count",
]

TRAIN_START = pd.Timestamp("2000-01-01")
TRAIN_END   = pd.Timestamp("2022-12-31")
SCORE_START = pd.Timestamp("2023-01-01")
SCORE_END   = pd.Timestamp("2024-12-31")

RESAMPLE_RULE = "M"     # month-end (canonical pandas alias)
MAX_K_FACTORS = len(INDICATORS)-1
FACTOR_ORDER  = 0
ERROR_COV     = "scalar"
TOP_PCT       = 0.05

In [67]:
# Optionally stabilize heavy-tailed counts (leave empty to keep raw)
LOG1P_VARS: list[str] = []  # e.g., ["Count of Citing Patents", "Claims Count", "IPC Count"]

In [68]:
# ---------- UTILS ----------

In [69]:
def winsorize(s: pd.Series, p: float = 0.01) -> pd.Series:
    # Safe quantiles even with all-NaN/constant series
    if s.notna().sum() == 0:
        return s
    lo, hi = s.quantile(p), s.quantile(1 - p)
    if pd.isna(lo) or pd.isna(hi) or lo == hi:
        return s
    return s.clip(lo, hi)

In [70]:
def robust_standardize_train(ts_train: pd.DataFrame) -> Tuple[pd.DataFrame, pd.Series, pd.Series]:
    mean = ts_train.mean(skipna=True)
    std = ts_train.std(ddof=1, skipna=True).replace(0, np.nan)
    return (ts_train - mean) / std, mean, std

In [71]:
def try_fit_dfa(mod: sm.tsa.DynamicFactor) -> Any:
    start_params = None
    # EM init if available
    try:
        if hasattr(mod, "fit_em"):
            res0 = mod.fit_em(maxiter=100, disp=False)
            start_params = getattr(res0, "params", None)
    except Exception:
        start_params = None
    # Model-provided starts
    if start_params is None:
        try:
            start_params = mod.start_params
        except Exception:
            start_params = None
    # Optimizer cascade
    for method, kw in [
        ("lbfgs", dict(maxiter=2000)),
        ("powell", dict(maxiter=400)),
        ("nm", dict(maxiter=2000)),
    ]:
        try:
            return mod.fit(start_params=start_params, method=method, disp=False, **kw)
        except Exception:
            continue
    # Last-ditch: powell then lbfgs with its params
    try:
        res1 = mod.fit(start_params=start_params, method="powell", maxiter=400, disp=False)
        return mod.fit(start_params=getattr(res1, "params", None), method="lbfgs", maxiter=2000, disp=False)
    except Exception as e:
        raise RuntimeError(f"DFA fit failed with all methods: {e}")

In [72]:
def _safe_converged(res: Any) -> bool:
    ret = getattr(res, "mle_retvals", None)
    if isinstance(ret, dict) and "converged" in ret:
        return bool(ret["converged"])
    return bool(getattr(res, "converged", False))

In [73]:
def _load_name_candidates(f_idx: int, var_idx: int, var_name: str | None) -> list[str]:
    # Common historical parameter name formats across statsmodels versions
    cands = [
        f"loading.f{f_idx}.y{var_idx}",
        f"loading.f{f_idx}.{var_idx}",
        f"loading.L[{var_idx-1},{f_idx-1}]",
    ]
    if var_name:
        cands += [
            f"loading.f{f_idx}.{var_name}",
            f"loading.f{f_idx}.y{var_name}",
            f"loading.{var_name}.f{f_idx}",
        ]
    return cands

In [74]:
def extract_loadings(res: Any, indicators: list[str], k: int, endog_names: list[str]) -> np.ndarray:
    # Map parameter name -> value robustly
    param_names = list(getattr(res, "param_names", []))
    params = getattr(res, "params", None)
    if params is None or len(param_names) != len(params):
        try:
            s = res.params  # pandas Series in some versions
            param_names = list(s.index)
            params = s.values
        except Exception:
            pass
    if params is None:
        raise RuntimeError("Could not access fitted parameter vector.")
    name2param = {pn: float(params[i]) for i, pn in enumerate(param_names)}

    load_mat = np.zeros((len(indicators), k), dtype=float)
    for f in range(1, k + 1):
        for i, _col in enumerate(indicators, start=1):
            var_name = None
            try:
                if isinstance(endog_names, (list, tuple)) and len(endog_names) >= i:
                    var_name = endog_names[i - 1]
            except Exception:
                var_name = None
            candidates = _load_name_candidates(f, i, var_name)
            val = None
            for cname in candidates:
                if cname in name2param:
                    val = name2param[cname]
                    break
            if val is None:
                # Fuzzy fallback
                prefix = f"loading.f{f}"
                matches = [kname for kname in name2param
                           if kname.startswith(prefix) and (
                               kname.endswith(f".y{i}") or
                               kname.endswith(f".{i}") or
                               (var_name and var_name in kname)
                           )]
                if matches:
                    val = name2param[matches[0]]
            if val is None:
                raise RuntimeError(f"Missing loading for factor {f}, variable index {i} ({_col}).")
            load_mat[i - 1, f - 1] = val
    return load_mat

In [75]:
def rowwise_weighted_sum_nan_safe(Z: np.ndarray, weights: np.ndarray) -> np.ndarray:
    W = weights.reshape(1, -1)
    mask = ~np.isnan(Z)
    numer = np.nan_to_num(Z) * W
    denom = (W * mask).sum(axis=1, keepdims=True)
    denom[denom == 0] = np.nan
    return (numer.sum(axis=1, keepdims=True) / denom).ravel()

In [76]:
# ---------- MAIN PIPELINE ----------

In [77]:
df = pd.read_csv(CSV_PATH)
df[DATE_COL] = pd.to_datetime(df[DATE_COL], errors="coerce")
# df = df.dropna(subset=[DATE_COL])

In [78]:
for c in INDICATORS:
    df[c] = pd.to_numeric(df[c], errors="coerce")

In [79]:
print("date range:", df[DATE_COL].min(), "->", df[DATE_COL].max())
print("na ratio per column:\n", df[INDICATORS].isna().mean().sort_values(ascending=False))

date range: 2000-05-15 00:00:00 -> 2024-12-31 00:00:00
na ratio per column:
 DWPI Count of Family Members              0.0
DWPI Count of Family Countries/Regions    0.0
Assignee Count                            0.0
Inventor Count                            0.0
Claims Count                              0.0
Legal Years Remaining                     0.0
IPC Count                                 0.0
dtype: float64


In [80]:
monthly = (df
           .set_index(DATE_COL)
           .sort_index()[INDICATORS]
           .resample(RESAMPLE_RULE)
           .sum(min_count=1))

ts = monthly.loc[TRAIN_START:TRAIN_END]

/var/folders/f1/zsswp6xx6bb2_yfyprkybnfc0000gn/T/ipykernel_24107/2372555477.py:4: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  .resample(RESAMPLE_RULE)


In [81]:
# Light imputation to avoid full-row drops:
ts_imp = ts.copy()
# Carry small gaps; leave long gaps as NaN
ts_imp = ts_imp.ffill(limit=2).bfill(limit=2)

In [82]:
med = ts_imp.median(skipna=True)
iqr = ts_imp.quantile(0.75) - ts_imp.quantile(0.25)
scale = iqr.replace(0, np.nan)
ts_train_std = (ts_imp - med) / scale

In [83]:
print("train shape:", ts_train_std.shape)
print("rows all-na:", int(ts_train_std.isna().all(axis=1).sum()), "of", len(ts_train_std))
print("cols all-na:", list(ts_train_std.columns[ts_train_std.isna().all()]))

train shape: (272, 7)
rows all-na: 0 of 272
cols all-na: []


In [84]:
ts_train_std = ts_train_std.dropna(how="all")

if len(ts_train_std) < (FACTOR_ORDER + MAX_K_FACTORS + 5):
    raise ValueError("Not enough timesteps after cleaning. Reduce k_factors or widen TRAIN window.")

In [85]:
ts_train = ts.loc[TRAIN_START:TRAIN_END]
if ts_train.shape[0] < 18:
    raise ValueError(f"Too few monthly periods in training window {TRAIN_START.date()}–{TRAIN_END.date()}.")

In [86]:
def try_fit_dfa(mod):
    # Fit with missing='drop' so the Kalman filter ignores NA entries rather than you pre-dropping entire rows
    return mod.fit(
        disp=False,
        maxiter=2000,
        cov_type="opg",
        enforce_stationarity=True,
        enforce_invertibility=True,  # pass here, not in constructor
        missing="drop"
    )

bic, best = {}, {}
for k in range(1, MAX_K_FACTORS + 1):
    # Remove invalid kw from constructor
    mod = sm.tsa.DynamicFactor(
        endog=ts_train_std,
        k_factors=k,
        factor_order=FACTOR_ORDER,
        error_cov_type=ERROR_COV,
    )
    try:
        res = try_fit_dfa(mod)
        if res.mle_retvals.get("converged", False) and np.isfinite(getattr(res, "bic", np.inf)):
            bic[k] = float(res.bic)
            best[k] = res
            print(f"[INFO] k={k}: BIC={res.bic:.2f}, converged=True")
        else:
            print(f"[WARN] k={k}: skipped (converged={res.mle_retvals.get('converged', False)}, finite_bic={np.isfinite(getattr(res,'bic', np.inf))})")
    except Exception as e:
        print(f"[WARN] k={k}: fitting failed -> {e}")

if not bic:
    raise RuntimeError("All candidate DFA fits failed. Check data span, NA rate, and k_factors.")

k_star = min(bic, key=bic.get)
res_best = best[k_star]
print("Selected k =", k_star)

/Users/tanishq/PycharmProjects/project1/.venv/lib/python3.12/site-packages/statsmodels/base/optimizer.py:21: FutureWarning: Keyword arguments have been passed to the optimizer that have no effect. The list of allowed keyword arguments for method lbfgs is: m, pgtol, factr, maxfun, epsilon, approx_grad, bounds, loglike_and_score, iprint. The list of unsupported keyword arguments passed include: missing. After release 0.14, this will raise.
  warnings.warn(


[INFO] k=1: BIC=5767.19, converged=True

/Users/tanishq/PycharmProjects/project1/.venv/lib/python3.12/site-packages/statsmodels/base/optimizer.py:21: FutureWarning: Keyword arguments have been passed to the optimizer that have no effect. The list of allowed keyword arguments for method lbfgs is: m, pgtol, factr, maxfun, epsilon, approx_grad, bounds, loglike_and_score, iprint. The list of unsupported keyword arguments passed include: missing. After release 0.14, this will raise.
  warnings.warn(



[INFO] k=2: BIC=4959.07, converged=True


/Users/tanishq/PycharmProjects/project1/.venv/lib/python3.12/site-packages/statsmodels/base/optimizer.py:21: FutureWarning: Keyword arguments have been passed to the optimizer that have no effect. The list of allowed keyword arguments for method lbfgs is: m, pgtol, factr, maxfun, epsilon, approx_grad, bounds, loglike_and_score, iprint. The list of unsupported keyword arguments passed include: missing. After release 0.14, this will raise.
  warnings.warn(


...

/Users/tanishq/PycharmProjects/project1/.venv/lib/python3.12/site-packages/statsmodels/base/optimizer.py:21: FutureWarning: Keyword arguments have been passed to the optimizer that have no effect. The list of allowed keyword arguments for method lbfgs is: m, pgtol, factr, maxfun, epsilon, approx_grad, bounds, loglike_and_score, iprint. The list of unsupported keyword arguments passed include: missing. After release 0.14, this will raise.
  warnings.warn(
/Users/tanishq/PycharmProjects/project1/.venv/lib/python3.12/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/Users/tanishq/PycharmProjects/project1/.venv/lib/python3.12/site-packages/statsmodels/base/optimizer.py:21: FutureWarning: Keyword arguments have been passed to the optimizer that have no effect. The list of allowed keyword arguments for method lbfgs is: m, pgtol, factr, maxfun, eps

[WARN] k=4: skipped (converged=False, finite_bic=True)


/Users/tanishq/PycharmProjects/project1/.venv/lib/python3.12/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/Users/tanishq/PycharmProjects/project1/.venv/lib/python3.12/site-packages/statsmodels/base/optimizer.py:21: FutureWarning: Keyword arguments have been passed to the optimizer that have no effect. The list of allowed keyword arguments for method lbfgs is: m, pgtol, factr, maxfun, epsilon, approx_grad, bounds, loglike_and_score, iprint. The list of unsupported keyword arguments passed include: missing. After release 0.14, this will raise.
  warnings.warn(


[WARN] k=5: skipped (converged=False, finite_bic=True)
[WARN] k=6: skipped (converged=False, finite_bic=True)
Selected k = 3


/Users/tanishq/PycharmProjects/project1/.venv/lib/python3.12/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


In [87]:
# # --- Model selection by BIC ---
# bic, best = {}, {}
# for k in range(1, MAX_K_FACTORS + 1):
#     mod = sm.tsa.DynamicFactor(
#         ts_train_std,
#         k_factors=k,
#         factor_order=FACTOR_ORDER,
#         error_cov_type=ERROR_COV,
#         enforce_stationarity=True,
#         enforce_invertibility=True,
#     )
#     try:
#         res = try_fit_dfa(mod)
#         converged = _safe_converged(res)
#         rbic = getattr(res, "bic", np.inf)
#         finite_bic = np.isfinite(rbic)
#         if converged and finite_bic:
#             bic[k] = float(rbic)
#             best[k] = res
#             print(f"[INFO] k={k}: BIC={rbic:.2f}, converged={converged}")
#         else:
#             print(f"[WARN] k={k}: skipped (converged={converged}, finite_bic={finite_bic})")
#     except Exception as e:
#         print(f"[WARN] k={k}: fitting failed -> {e}")
#
# if not bic:
#     raise RuntimeError("All candidate DFA fits failed or did not converge with finite BIC. "
#                        "Action: reduce MAX_K_FACTORS, simplify model, or lengthen TRAIN window.")

In [88]:
optimal_k = min(bic, key=bic.get)
res = best[optimal_k]
print(f"[OK] Selected k={optimal_k} with BIC={bic[optimal_k]:.2f}")

[OK] Selected k=3 with BIC=4230.11


In [89]:
# --- Loadings -> indicator weights ---
try:
    endog_names = list(getattr(res.model, "endog_names", []))
    if isinstance(endog_names, str):
        endog_names = [endog_names]
except Exception:
    endog_names = []
load_mat = extract_loadings(res, INDICATORS, optimal_k, endog_names=endog_names)

var_contrib = (load_mat ** 2).sum(axis=1)
total = var_contrib.sum()
if not np.isfinite(total) or total <= 0:
    raise RuntimeError("Invalid variance contributions from loadings.")
weights = var_contrib / total

/var/folders/f1/zsswp6xx6bb2_yfyprkybnfc0000gn/T/ipykernel_24107/4098707077.py:14: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  name2param = {pn: float(params[i]) for i, pn in enumerate(param_names)}


In [90]:
# --- Export weights (all 10 indicators) ---
weights_df = pd.DataFrame({
    "Indicator": INDICATORS,
    "VarianceContribution": var_contrib,
    "Weight": weights
}).sort_values("Weight", ascending=False).reset_index(drop=True)

In [91]:
weights_df

,Indicator,VarianceContribution,Weight
0,DWPI Count of Family Countries/Regions,68.290157,0.294948
1,DWPI Count of Family Members,35.682777,0.154116
2,Legal Years Remaining,32.616530,0.140872
3,Inventor Count,31.540156,0.136223
4,Claims Count,25.335412,0.109425
5,Assignee Count,19.296404,0.083342
6,IPC Count,18.771136,0.081073


In [92]:
# --- Scoring 2023–2024 on TRAIN scale (row-wise) ---
df_train_rows = df[df[DATE_COL].between(TRAIN_START, TRAIN_END, inclusive="both")]
train_row_mean = df_train_rows[INDICATORS].mean(skipna=True)
train_row_std  = df_train_rows[INDICATORS].std(ddof=1, skipna=True).replace(0, np.nan)

df_eval = df[df[DATE_COL].between(SCORE_START, SCORE_END, inclusive="both")].copy()
if df_eval.empty:
    raise ValueError(f"No rows in scoring window {SCORE_START.date()}–{SCORE_END.date()}.")

Zcols = []
for c in INDICATORS:
    z = (df_eval[c].astype(float) - train_row_mean[c]) / train_row_std[c]
    zname = f"Z::{c}"
    df_eval[zname] = z
    Zcols.append(zname)

Z = df_eval[Zcols].to_numpy(dtype=float)
qi = rowwise_weighted_sum_nan_safe(Z, weights)
df_eval["QualityIndex"] = qi

df_eval = df_eval.sort_values("QualityIndex", ascending=False).reset_index(drop=True)
k_top = int(np.ceil(TOP_PCT * len(df_eval))) if len(df_eval) else 0
df_eval["Top5pct"] = False
if k_top > 0:
    df_eval.loc[:k_top - 1, "Top5pct"] = True

In [93]:
# --- Save & print summaries ---
train_tag = f"{TRAIN_START.year}_{TRAIN_END.year}"
weights_path = f"dfa_weights_{train_tag}.csv"
scores_path  = "quality_scores_2023_24.csv"

weights_df.to_csv(weights_path, index=False)
df_eval[[ID_COL, DATE_COL, "QualityIndex", "Top5pct"]].to_csv(scores_path, index=False)

print("\n=== DFA Summary ===")
print(f"Train window: {TRAIN_START.date()} to {TRAIN_END.date()}")
print(f"Score window: {SCORE_START.date()} to {SCORE_END.date()}")
print(f"Candidates tried (k): {list(bic.keys())}")
print(f"Optimal k: {optimal_k}  |  Converged: {_safe_converged(res)}")
print(f"Saved weights -> {os.path.abspath(weights_path)}")
print(f"Saved scores  -> {os.path.abspath(scores_path)}")

print("\nIndicator weights (variance share; higher = more influence):")
print(weights_df.to_string(index=False))

print("\nWeights in original indicator order:")
for ind, w in zip(INDICATORS, weights):
    print(f"{ind}: {w:.6f}")

print("\nTop 10 by QualityIndex:")
print(df_eval[[ID_COL, "QualityIndex", "Top5pct"]].head(10).to_string(index=False))


=== DFA Summary ===
Train window: 2000-01-01 to 2022-12-31
Score window: 2023-01-01 to 2024-12-31
Candidates tried (k): [1, 2, 3]
Optimal k: 3  |  Converged: True
Saved weights -> /Users/tanishq/PycharmProjects/NIScPR/FinTech/DerwentData_TS/dfa/7 indicators/dfa_weights_2000_2022.csv
Saved scores  -> /Users/tanishq/PycharmProjects/NIScPR/FinTech/DerwentData_TS/dfa/7 indicators/quality_scores_2023_24.csv

Indicator weights (variance share; higher = more influence):
                             Indicator  VarianceContribution   Weight
DWPI Count of Family Countries/Regions             68.290157 0.294948
          DWPI Count of Family Members             35.682777 0.154116
                 Legal Years Remaining             32.616530 0.140872
                        Inventor Count             31.540156 0.136223
                          Claims Count             25.335412 0.109425
                        Assignee Count             19.296404 0.083342
                             IPC Count   